### ML notebook

In [1]:
n_samples = 10000
gens_per_ideal = 2
set_random_seed()

import random

random.seed(6)

In [2]:
# P.<x,y> = PolynomialRing(QQ, 2, order='deglex')
# P.<x,y> = PolynomialRing(QQ, 2, order='lex')
# P.<x,y> = PolynomialRing(QQ, 2, order='degrevlex')
# P.<x,y> = PolynomialRing(QQ, 2, order=TermOrder('wdegrevlex',(1,3)))
# P.<x,y> = PolynomialRing(QQ, 2, order=TermOrder('negwdegrevlex',(1,3)))

#### Functions to create features and label

In [3]:
def is_GB_worth(GB): # Receives a GB
    n_depend1=0
    n_depend2=0
    for pol in GB:
        depend_x=False
        depend_y=False
        
        # Comprobamos si depende de x y de y
        if pol.degree(x) > 0:
            depend_x=True
        if pol.degree(y) > 0:
            depend_y=True
            
        # Actualizamos el numero de de polinomios que dependen de cada cosa
        if depend_x and depend_y:
            n_depend2 += 1
        elif depend_x or depend_y:
            n_depend1 += 1
    if n_depend1 >= n_depend2:
        return True
    else:
        return False
    
def terms_on_x(gens):
    count=0
    for item in gens:
        for exp in item.exponents():
            if exp[0] > 0:
                count+=1
    return count
        
def terms_on_y(gens):
    count=0
    for item in gens:
        for exp in item.exponents():
            if exp[1] > 0:
                count+=1
    return count

def total_terms_diff(gens):
    total=0
    
    for i in range(len(gens)):
        n_terms=len(gens[i].exponents())
        
        if i%2 == 0:
            total+=n_terms
        else:
            total-=n_terms
    
    return abs(total)

def total_degs_diff(gens):
    total = 0
    
    for i in range(len(gens)):
        total_deg_item=0
        for exp in gens[i].exponents():
            aux=exp[0]+exp[1]
            if total_deg_item < aux:
                total_deg_item=aux
        
        if i%2 == 0:
            total+= total_deg_item
        else:
            total-= total_deg_item
            
    return abs(total)
    

In [4]:
from collections import defaultdict

def n_homogeneous_comps(pol):
    dic = defaultdict(pol.parent())
    
    for coeff,monom in pol:
        dic[monom.degree()] += coeff * monom
        
    return len(dic)

#### Voy a generar una lista de ideales para los polinomios generados aleatoriamente y con información adicional

In [5]:
def do_experiment():
    lstOfIdeals = []
    isWorthCount = 0
    for i in range(n_samples):
        generators = []
        n_homogeneous = 0
        total_homogeneous_comps=0

        for j in range(gens_per_ideal):
            pol_degree=randint(2, 10)
            p = P.random_element(pol_degree)

            # homogeneous components for the given polynomial
            total_homogeneous_comps += n_homogeneous_comps(p)

            generators.append(p)
            if p.is_homogeneous():
                n_homogeneous += 1

        # Definimos el ideal a partir de los generadores anteriores
        I = ideal(generators)
        B = I.groebner_basis()
        isWorth = is_GB_worth(B)
        
        if isWorth:
            isWorthCount+=1
        
        lstOfIdeals.append([I, 
                            n_homogeneous,
                            total_degs_diff(generators),
                            total_terms_diff(generators),
                            terms_on_x(generators), 
                            terms_on_y(generators), 
                            terms_on_x(generators) - terms_on_y(generators),
                            total_homogeneous_comps, 
                            isWorth])
        
    print("Worth it bases in the given dataset "+str(isWorthCount))
        
    import numpy as np

    X = np.array([[item[1],item[2],item[3],item[4],item[5],item[6],item[7]] for item in lstOfIdeals])
    y = np.array([item[8] for item in lstOfIdeals])

    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    
    from sklearn import svm
    
    classifier = svm.SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
                  decision_function_shape='ovr', degree=3, gamma=0.0100000000000000,
                  kernel='rbf', max_iter=-1, probability=False, random_state=None,
                  shrinking=True, tol=0.001, verbose=False)
    
    classifier.fit(X_train, y_train)
    
    preds = classifier.predict(X_test)
    
    hits=0
    fails=0
    for i in range(len(y_test)):
        if y_test[i] == preds[i]:
            hits+=1
        else:
            fails+=1
    
    from sklearn.metrics import classification_report, confusion_matrix
    
    print("Classification report:")
    print(classification_report(y_test,preds))
    
    return hits/(hits+fails) # Return score (between 0 and 1)

In [6]:
iters=1
t_score=0.0

for i in range(iters):
    P.<x,y> = PolynomialRing(QQ, 2, order='lex')
    print('INFO: iter '+str(i+1))
    t_score += do_experiment()
    
avg_score=t_score/iters

INFO: iter 1
Worth it bases in the given dataset 5094
Classification report:
              precision    recall  f1-score   support

       False       0.65      0.70      0.67       968
        True       0.70      0.65      0.67      1032

   micro avg       0.67      0.67      0.67      2000
   macro avg       0.67      0.67      0.67      2000
weighted avg       0.67      0.67      0.67      2000



In [7]:
avg_score

0.673000000000000

### The code below is to perform a grid search to find the best hyperparameters

In [8]:
param_grid = {'C': [0.1,1,10,100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf']}

In [9]:
P.<x,y> = PolynomialRing(QQ, 2, order='lex')
lstOfIdeals = []
isWorthCount = 0
for i in range(n_samples):
    generators = []
    n_homogeneous = 0
    total_homogeneous_comps=0

    for j in range(gens_per_ideal):
        pol_degree=randint(2, 10)
        p = P.random_element(pol_degree)

        # homogeneous components for the given polynomial
        total_homogeneous_comps += n_homogeneous_comps(p)

        generators.append(p)
        if p.is_homogeneous():
            n_homogeneous += 1

    # Definimos el ideal a partir de los generadores anteriores
    I = ideal(generators)
    B = I.groebner_basis()
    isWorth = is_GB_worth(B)

    if isWorth:
        isWorthCount+=1

    lstOfIdeals.append([I, 
                        n_homogeneous,
                        total_degs_diff(generators),
                        total_terms_diff(generators),
                        terms_on_x(generators), 
                        terms_on_y(generators), 
                        terms_on_x(generators) - terms_on_y(generators),
                        total_homogeneous_comps, 
                        isWorth])

print("Worth it bases in the given dataset "+str(isWorthCount))

import numpy as np

X = np.array([[item[1],item[2],item[3],item[4],item[5],item[6],item[7]] for item in lstOfIdeals])
y = np.array([item[8] for item in lstOfIdeals])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

print("Best estimator")
print(grid.best_estimator_)

preds = grid.predict(X_test)

print("Confunsion matrix and report:")
print(confusion_matrix(y_test,preds))
print(classification_report(y_test,preds))

hits=0
fails=0
for i in range(len(y_test)):
    if y_test[i] == preds[i]:
        hits+=1
    else:
        fails+=1

avg_score=t_score/iters
avg_score

Worth it bases in the given dataset 5122
Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] kernel=rbf, C=0.100000000000000, gamma=1 ........................


/home/santi/SageMath/local/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ......... kernel=rbf, C=0.100000000000000, gamma=1, total=   1.3s
[CV] kernel=rbf, C=0.100000000000000, gamma=1 ........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV] ......... kernel=rbf, C=0.100000000000000, gamma=1, total=   1.3s
[CV] kernel=rbf, C=0.100000000000000, gamma=1 ........................
[CV] ......... kernel=rbf, C=0.100000000000000, gamma=1, total=   1.3s
[CV] kernel=rbf, C=0.100000000000000, gamma=0.100000000000000 ........
[CV]  kernel=rbf, C=0.100000000000000, gamma=0.100000000000000, total=   1.2s
[CV] kernel=rbf, C=0.100000000000000, gamma=0.100000000000000 ........
[CV]  kernel=rbf, C=0.100000000000000, gamma=0.100000000000000, total=   1.2s
[CV] kernel=rbf, C=0.100000000000000, gamma=0.100000000000000 ........
[CV]  kernel=rbf, C=0.100000000000000, gamma=0.100000000000000, total=   1.2s
[CV] kernel=rbf, C=0.100000000000000, gamma=0.0100000000000000 .......
[CV]  kernel=rbf, C=0.100000000000000, gamma=0.0100000000000000, total=   1.2s
[CV] kernel=rbf, C=0.100000000000000, gamma=0.0100000000000000 .......
[CV]  kernel=rbf, C=0.100000000000000, gamma=0.0100000000000000, total=   1.2s
[CV] kernel=rbf, C=0.100000000000000, ga

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  1.6min finished


Best estimator
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0100000000000000,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Confunsion matrix and report:
[[623 367]
 [298 712]]
              precision    recall  f1-score   support

       False       0.68      0.63      0.65       990
        True       0.66      0.70      0.68      1010

   micro avg       0.67      0.67      0.67      2000
   macro avg       0.67      0.67      0.67      2000
weighted avg       0.67      0.67      0.67      2000



0.673000000000000

In [10]:
param_grid = {'C': [1,2e5,2e6,2e7,2e8,2e9,2e10],
              'gamma': [2e-10,2e-9,2e-8,2e-7,2e-6,2e-5,2e-2],
              'kernel': ['rbf']}

In [ ]:
P.<x,y> = PolynomialRing(QQ, 2, order='lex')
lstOfIdeals = []
isWorthCount = 0
for i in range(n_samples):
    generators = []
    n_homogeneous = 0
    total_homogeneous_comps=0

    for j in range(gens_per_ideal):
        pol_degree=randint(2, 10)
        p = P.random_element(pol_degree)

        # homogeneous components for the given polynomial
        total_homogeneous_comps += n_homogeneous_comps(p)

        generators.append(p)
        if p.is_homogeneous():
            n_homogeneous += 1

    # Definimos el ideal a partir de los generadores anteriores
    I = ideal(generators)
    B = I.groebner_basis()
    isWorth = is_GB_worth(B)

    if isWorth:
        isWorthCount+=1

    lstOfIdeals.append([I, 
                        n_homogeneous,
                        total_degs_diff(generators),
                        total_terms_diff(generators),
                        terms_on_x(generators), 
                        terms_on_y(generators), 
                        terms_on_x(generators) - terms_on_y(generators),
                        total_homogeneous_comps, 
                        isWorth])

print("Worth it bases in the given dataset "+str(isWorthCount))

import numpy as np

X = np.array([[item[1],item[2],item[3],item[4],item[5],item[6],item[7]] for item in lstOfIdeals])
y = np.array([item[8] for item in lstOfIdeals])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

print("Best estimator")
print(grid.best_estimator_)

preds = grid.predict(X_test)

print("Confunsion matrix and report:")
print(confusion_matrix(y_test,preds))
print(classification_report(y_test,preds))

hits=0
fails=0
for i in range(len(y_test)):
    if y_test[i] == preds[i]:
        hits+=1
    else:
        fails+=1

avg_score=t_score/iters
avg_score

Worth it bases in the given dataset 5123
Fitting 3 folds for each of 49 candidates, totalling 147 fits
[CV] kernel=rbf, C=1, gamma=2.00000000000000e-10 .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... kernel=rbf, C=1, gamma=2.00000000000000e-10, total=   1.2s
[CV] kernel=rbf, C=1, gamma=2.00000000000000e-10 .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV] ...... kernel=rbf, C=1, gamma=2.00000000000000e-10, total=   1.2s
[CV] kernel=rbf, C=1, gamma=2.00000000000000e-10 .....................
[CV] ...... kernel=rbf, C=1, gamma=2.00000000000000e-10, total=   1.2s
[CV] kernel=rbf, C=1, gamma=2.00000000000000e-9 ......................
[CV] ....... kernel=rbf, C=1, gamma=2.00000000000000e-9, total=   1.3s
[CV] kernel=rbf, C=1, gamma=2.00000000000000e-9 ......................
[CV] ....... kernel=rbf, C=1, gamma=2.00000000000000e-9, total=   1.3s
[CV] kernel=rbf, C=1, gamma=2.00000000000000e-9 ......................
[CV] ....... kernel=rbf, C=1, gamma=2.00000000000000e-9, total=   1.3s
[CV] kernel=rbf, C=1, gamma=2.00000000000000e-8 ......................
[CV] ....... kernel=rbf, C=1, gamma=2.00000000000000e-8, total=   1.3s
[CV] kernel=rbf, C=1, gamma=2.00000000000000e-8 ......................
[CV] ....... kernel=rbf, C=1, gamma=2.00000000000000e-8, total=   1.3s
[CV] kernel=rbf, C=1, gamma=2.00000000000000e-8 ......................
[CV] .